In [1]:
from pyspark.sql import SparkSession
import time

#Start stopwatch
startTime = time.time()

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.1.153:7077") \
        .appName("group_09_fullDataset")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.driver.memory","8G")\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",8)\
        .getOrCreate()
    
        
# Old API (RDD
spark_context = spark_session.sparkContext

In [ ]:
import h5py
import sys
import io

#rdd = spark_context.binaryFiles("hdfs://192.168.1.153:9000/millionsongs/data/*/*/*/*")
rdd = spark_context.binaryFiles("/mnt/ms/data/*/*/*/*")
#rdd = rdd.repartition(30)

     # read a dataset and return it as a Python list #

def f(x):
    # return print(sys.path)
    
    # x[0] = filename
    # x[1] = binary content
    with h5py.File(io.BytesIO(x[1])) as f:
        # drill down with a path
        #return list(f.keys())
        g = f['analysis']['songs']['duration']
        # g is a 'dataset'
        return list(g)

rdd = rdd.map(f).cache()

In [ ]:
def time_conversion(duration):
    duration_in_half_minutes = duration[0]/30
    discretitized_duration = round(duration_in_half_minutes)
    discretized_duration_in_minutes = discretitized_duration/2
    return discretized_duration_in_minutes

converted_rdd = rdd.map(time_conversion)
print(converted_rdd.take(10))
print(converted_rdd.count())

In [ ]:
result = converted_rdd.map(lambda time: (time, 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[0], True)
print(result.take(10))
endTime = time.time()
print("Time Elapsed {}".format((endTime-startTime)/60))

In [ ]:
import matplotlib.pyplot as plt
plt.bar(*zip(*result.collect()))
plt.show